## Javascript promises and Async programming

### Fetching data
* one way to make sure the order of function calls is to use pyramid structure of functions, as shown below:
```javascript
    a("test", (err, aResult) => {
      b(aResult, (err, bResult) => {
        c(bResult, (err, cResult) => {
          d(cResult);
        });
      });
    });

```
  + a executes before b, which executes before c, which executes before d 
  + dirty code
  + error handling difficult
* promise
  + an object that represents the eventual completion (or failure) of an asynchronous operation, and its resulting value
  + readable asynchronous code
* promise states
  + pending
  + fulfilled (return a single value)
  + rejected (when a asynchronous call has failed)
    + it returns with the reason why it failed similar to the catch function in try/catch block
* code example
  + axios send HTTP GET request to the url, when sends back results as a Promise
  + the get function is chained to then function which will be called once the get function get the Promise object with request fulfilled
  + in then function, we first desctruct the return JSON object and get data property, then use the retrieved data object to set the text in result box
  + if the promise is rejected, then the error will be sent to catch function (the only object sent to catch is the error)
  + note that a fulfilled promise will send the entire result obj to then function, that is why we need to destructure the data property, but the rejected promise only sends the error to catch function
  
```javascript

export function get() {
    axios.get("http://localhost:3000/orders/1")
    .then(({data}) => {
        setText(JSON.stringify(data));
    })
    .catch(err => setText(err)); 
}
```

### chaining Promises
* both then and catch functions return Promises (Promises return Promises). This allows us to chain Promises together
* in the following code example
  + the first then function set the text, and then return "Pluralsight" (you can return anything)
  + this string is wrapped as a Promise object and returned by then
  + the returned Promise object is then console logged
```javascript
  axios.get("http://localhost:3000/orders/1").then(({ data }) => {
      setText(JSON.stringify(data));
      return "Pluralsight";
  })
  .then(result => console.log(result))
```

* let's chain two HTTP GET requests together (remember to add return in the first then clause
  + the catch function will catch any errors in either of the two then functions
```javascript
  axios.get("http://localhost:3000/orders/1").then(({ data }) => {
      return axios.get(`http://localhost:3000/addresses/${data.shippingAddress}`
     })
  .then(({data}) => {
    setText(`City: ${data.city}`);
  })
  .catch(err => setText(err));
}
```

* if we have two requests, how to make sure the errors specific to each then function will be caught separately, rather than being caught by the second catch function?
  + we can put a catch function for each of the then function
  + here the second catch function will catch the error from both the first catch and second then functions
```javascript
  axios.get("http://localhost:3000/orders/1").then(({ data }) => {
      return axios.get(`http://localhost:3000/addresses/${data.shippingAddress}`
     })
  .catch(err => {
      setText(err);
      return {data:{}};
  })
  .then(({data}) => {
    setText(`City: ${data.city}`);
  })
  .catch(err => setText(err));
}
```  
* run code after either then and catch (in this example, to hide the waiting sign
```javascript
  axios.get("http://localhost:3000/orders/1").then(({ data }) => {
      return axios.get(`http://localhost:3000/addresses/${data.shippingAddress}`
     })
  .then(({data}) => {
    setText(`City: ${data.city}`);
  })
 .catch(err => setText(err))
 .finally(() => {
      setTimeout(() => {
          hideWaiting();
      }, 1500);
  });
}
```  

### Creating promises
* a pending promise is a promise that has not yet settled
* Promise
  + object that represents the eventual completion (or failure) of an asynchronous operation, and its resulting value
* how to create a Promise
  + use new Promise, with a parameter called resolve
  + define the function body, and in resolve(), define the object to return when request is fulfilled
  + consume the Promise by then() function and you will see the "timeout!" text is the object returned from resolve
```javascript

export function timeout() {
    
    const wait = new Promise((resolve) => {
        setTimeout(() => {
            resolve("Timeout!");
        }, 1500);
    });

    wait.then(text => setText(text));
}

```

* what about functions that will be invoked multiple times by setInterval?
  + setInterval fires multiple times while setTimeout fires only once
  + you will only see the first execution of the function fired. Once a Promise is settled, it is done. The following firing of the resolve() function will not be executed, although console.log is successfully fired multiple times
  + the function defined in Promise still keeps running, but not the resolve() function
```javascript

export function interval() {
    let counter = 0;
    const wait = new Promise((resolve) => {
        setInterval(() => {
            console.log("INTERVAL");
            resolve(`Timeout! ${++counter}`);
        }, 1500);
    });

    wait.then(text => setText(text));
    .finally(() => appendText(` -- Done ${counter}`));
}

```  

   
* add both resolve and reject to handle fulfill and reject states
  + when creating a Promise
    + for fulfill state, return the object/data using resolve()
    + for reject state, return the error message/object uisng reject()
  + when consuming the Promise
    + use then and its parameter representing the object/data returned from resolve()
    + use catch and its parameter representing the error message/object returned from reject()
  
```javascript
export function xhr() {
    let request = new Promise((resolve, reject) => {
        let xhr = new XMLHttpRequest();
        xhr.open("GET", "http://localhost:3000/users/7");
        xhr.onload = () => {
            if (xhr.status === 200) {
                resolve(xhr.responseText);
            } else {
                reject(xhr.statusText);
            }
        }
        xhr.onerror = () => reject("Request Failed");
        xhr.send();
    });
    
    request.then(result=> setText(result))
    .catch(reason => setText(reasson));
}
```

* waiting for all promises to resolve using Promise.all()
  + call HTTP GET by axios
  + wait until all API asynchronous calls are returned by Promise.all(), and define the order to process these Promises
  + in then() define cat, stat and type Promise objects, and append the content to text by appendText
  + Promise.all() wait return when 
    + either all Promises are fulfilled
    + or one of them is rejected (the first rejected Promise will cause the function to return)
      + the returned rejected obj will be handled by catch()
```javascript
    export function allPromises() {
        let category = axios.get("http://localhost:3000/tiemCategories");
        let statuses = axios.get("http://localhost:3000/orderStatuses");
        let userTypes = axios.get("http://localhost:3000/userTypes");

        Promise.all([category, statuses, userTypes])
        .then(([cat, stat, type]) => {
            setText("");

            appendText(JSON.stringify(cat.data));
            appendText(JSON.stringify(stat.data));
            appendText(JSON.stringify(type.data));
        })
        .catch(err => setText(err));
    }
```

* what if we don't need to wait all the Promises to be resolved, if some of them are rejected, we can still proceed? We use Promises.allSettled
  + Promises.allSettled returns two JS object, one is resolved, the other is rejected
    + in resolved there are two properties: status: fulfilled and value: {}
    + in rejected, there are two properties: status: rejected and reason: {}
    + since both resolved and rejected are returned in one object, we only need to use .then to consume, and no need to use catch, but is recommended to catch other errors
    
    ```javascript
        // resolved.js 
        {
            status: "fulfilled",
                value: {}
        }

        // rejected.js
        {
            status: "rejected",
                reason: {}
        }

    ```
  + code example of using allSettled
  ```javascript
    export function allSettled() {
        let category = axios.get("http://localhost:3000/tiemCategories");
        let statuses = axios.get("http://localhost:3000/orderStatuses");
        let userTypes = axios.get("http://localhost:3000/userTypes");

        Promise.allSettled([category, statuses, userTypes])
        .then((values) => {
            let results = values.map(v => {
                if(v.status === 'fulfilled'){
                    return `FULFILLED: ${JSON.stringify(v.value.data[0])} `
                    }
                    
                return `REJECTED: ${v.reason.message} `;
                });

            setText(results);
         })
        .catch(err => setText(err));
    }
```

* racing Promises
  + if you have several API calls for the same information, but just need to get the fasted returned results and don't need to wait for other reponses, you can use Promise.race() function
  + race() stops when the first response is settled, which may be resolved or rejected. Even if the later response returns the resolved data, the function still directly goes to catch block, if the first reponse received is a rejected result.
  ```javascript 
  export function race() {
      let users = axios.get("http://localhost:3000/users");
      let backup = axios.get("http://localhost:3001/users");
      
      Promise.race([users, backup])
      .then(users => setText(JSON.stringify(users.data)))
      .catch(reason => setText(reason));
  } 

### Async/Await
* Async/await is syntactic sugar
  + async
    + designate that a function is asynchronous
    + the function returns a Promise that wrap whatever you return in the function
    
    ```javascript

    // async
    async function getNames() {
        return [];
    }

    const getNames = async () => {
        return [];
    }
```

  + await
    + pauses the execution of an asynchronous function while it waits for Promise to fulfill
    + can only be used inside an async function
    + only blocks the current function
    + it does not block calling function
    + in the following code example
      + await will pause the code inside getNames to wait until someFunc() returns, and therefore, delay the execution of doSomethingElse()
      + await will not impact getAddresses()
    
    ```javascript

        // await code example

        const getNames = async () => {
            await someFunc();
            doSomethingElse();
        }

        getNames();
        getAddresses();

    ```
  
* Use the standard try/catch to handle both synchronous and asynchronous code

```javascript

    export async function get() {
        try {
        const {data} = await axios.get("http://localhost:3000/orders/1");
        setText(JSON.stringify(data));
        } catch (error) {
            setText(error);
        }        
    }

    export async function get() {
        axios.get("http://localhost:3000/orders/1")
        .then(({data}) => {
        setText(JSON.stringify(data))
    })
     .catch(err => setText(err));
    }   

```

* chain async/await

```javascript

    export async function chain() {
        const {data} = await axios.get("http://localhost:3000/orders/1");

        // destructure data and assign it to address
        const {data: address} = await axios.get(`http://localhost:3000/addresses/${data.shippingAddress}`);

        setText(`City: ${JSON.stringify(address.city)}`);
    }

```

* await concurrent requests
  + this is done by first defining the functions as function expressions, and then call await. The two functions will be called at the same time
  + the following appendText calls will wait until both await functions return
```javascript

    export async function concurrent() {
        // define two functions first as function expressions
        const orderStatus = axios.get("http://localhost:3000/orderStatus");
        const orders = axios.get("http://localhost:3000/orders");

        setText("");
        
        const {data: statuses} = await orderStatus;
        const {data: order} = await orders;
        
        appendText(JSON.stringify(statuses));
        appendText(JSON.stringify(order[0]));
        
    }

```

* awaiting parallel calls
  + use the await decorated Promise.all, and list the async anonymous functions IIFEs to execute each sync calls simultaneously
   + immediately invoked function expression
```javascript

    export async function parallel() {
        setText("");
        
        await Promise.all([
            (async () => {
                const {data} = await axios.get("http://localhost:3000/orderStatus");
                appendText(JSON.stringify(data));
            })(),
            (async () => {
                const {data} = await axios.get("http://localhost:3000/orders");
                appendText(JSON.stringify(data));
            })()
            
        ]);
        
    }

```